<a href="https://colab.research.google.com/github/yulindeyo/dln_from_scratch3/blob/main/%EB%B0%91%EB%94%A53_2%EB%8B%A8%EC%9B%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2단원
- 다중 입력과 다중 출력인 함수도 처리할 수 있도록 확장
- 계산을 자연스러운 코드(+, * 등)로 표현할 수 있도록 수정
- 파이썬 패키지로 병합

In [1]:
# Step 10 까지 최종 코드
import numpy as np

class Variable:
    def __init__(self, data):
        if data is not None:                            # 데이터가 None은 아닌데
            if not isinstance(data, np.ndarray):        # ndarray도 아니면      오류 발생
                raise TypeError("{}은(는) 지원하지 않습니다.".format(type(data)))
        
        self.data = data
        self.grad = None 
        self.creator = None 

    def set_creator(self, func):
        self.creator = func
    
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        
        funcs = [self.creator]
        while funcs:
            f = funcs.pop() 
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)
                

def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x 

class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(as_array(y))
        output.set_creator(self)
        self.input = input
        self.output = output
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()

class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx


class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx


def square(x):
    return Square()(x)


def exp(x):
    return Exp()(x)

## Step 11
### 가변 길이 인수 (순전파 편)

In [2]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x 
    
# 기존 Function 클래스
class Function:
  def __call__(self, input):
    x = input.data                  # 1
    y = self.forward(x)             # 2
    output = Variable(as_array(y))  # 3
    output.set_creator(self)        # 4
    self.input = input
    self.output = output
    return output

  def forward(self, x):
    raise NotImplementedError()

  def backward(self, gy):
    raise NotImplementedError()

- 1. Function의 call 메서드는 Variable 이라는 '상자'에서 실제 데이터를 꺼낸다.
- 2. forward 메서드에서 구체적인 계산
- 3. 계산 결과를 Variable에 넣고
- 4. 자신이 'creator'라고 원산지 표시를 한다.

In [3]:
# 가변 길이 입출력을 고려한 Function 클래스
class Function:
  def __call__(self, inputs):
    xs = [input.data for input in inputs]                  # 1
    ys = self.forward(xs)                                  # 2
    outputs = [Variable(as_array(y)) for y in ys]          # 3

    for output in outputs:                                 # 4
      output.set_creator(self)                             # 4                             
    
    self.inputs = inputs
    self.outputs = outputs
    return outputs

  def forward(self, xs):
    raise NotImplementedError()

  def backward(self, gys):
    raise NotImplementedError()

In [4]:
class Add(Function):
  def forward(self, xs):
    x0, x1 = xs
    y = x0 + x1
    return (y,) # Function 클래스에서 inputs에 ys가 들어가기 때문에 리스트 혹은 튜플로 리턴해야함
  
xs = [Variable(np.array(2)), Variable(np.array(3))]
f = Add()
ys = f(xs) # ys 튜플
y = ys[0]
print(y.data)

# Add()를 사용할 때, 입력 값을 리스트로 주는 것이 번거로움
# y = ys[0]에서 리턴 값으로 튜플을 받는 것은 번거로움 

5


## Step 12
### 가변 길이 인수 (개선 편)

- 1. Add 클래스(혹은 다른 구체적인 함수 클래스)를 '사용하는 사람'을 위한 개선
- 2. '구현하는 사람'을 위한 개선


#### 12.1 첫 번째 개선: 함수를 사용하기 쉽게

```python
xs = [Variable(np.array(2)), 
      Variable(np.array(3))]
f = Add()
ys = f(xs)
y = ys[0]
```
기존 위의 코드를 아래와 같이 변경
```python
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
f = Add()
y = f(x0, x1)
```

In [5]:
# 가변 길이 입출력을 고려한 Function 클래스
class Function:
  def __call__(self, *inputs): # positional argument : 다중 입력을 자동으로 tuple로 변환해줌
    xs = [input.data for input in inputs]
    ys = self.forward(xs)
    outputs = [Variable(as_array(y)) for y in ys]

    for output in outputs:
      output.set_creator(self)                        
    
    self.inputs = inputs
    self.outputs = outputs
    return outputs if len(outputs)>1 else outputs[0] # 리스트의 원소가 하나 이상이면 모두 반환, 한개이면 0번째 반환

  def forward(self, xs):
    raise NotImplementedError()

  def backward(self, gys):
    raise NotImplementedError()

# 기존 Add 클래스 코드
class Add(Function):
  def forward(self, xs):
    x0, x1 = xs
    y = x0 + x1
    return (y,) # Function 클래스에서 inputs에 ys가 들어가기 때문에 리스트 혹은 튜플로 리턴해야함

x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
f = Add()
y = f(x0, x1)
print(y.data)

5


In [6]:
# positional argument
def a(*b):
  print(b)

a(1,2)
a(1)

(1, 2)
(1,)


#### 두 번째 개선: 함수를 구현하기 쉽도록
- Add 클래스를 '구현하는 사람'을 위한 개선

```python
class Add(Function):
  def forward(self, xs):
    x0, x1 = xs
    y = x0 + x1
    return(y,)
```
위의 코드를 아래와 같이 변경

```python
class Add(Function):
  def forward(self, x0, x1):
    y = x0 + x1
    return y
```

In [7]:
# 두 번째 개선을 위해 Function 클래스 수정
class Function:
  def __call__(self, *inputs):
    xs = [input.data for input in inputs]
    ys = self.forward(*xs) # postion argument로 unpacking
    if not isinstance(ys, tuple): # 리턴 타입이 튜블이 아닌 경우
      ys = (ys,) # 튜플로 변환
    outputs = [Variable(as_array(y)) for y in ys]

    for output in outputs:
      output.set_creator(self)                        
    
    self.inputs = inputs
    self.outputs = outputs
    return outputs if len(outputs)>1 else outputs[0]

  def forward(self, xs):
    raise NotImplementedError()

  def backward(self, gys):
    raise NotImplementedError()

# 두 번째 개선의 Add 클래스 수정
class Add(Function):
  def forward(self, x0, x1):
    y = x0 + x1
    return y

# Add 클래스를 함수로 사용하기 위한 코드
def add(x0, x1):
  return Add()(x0, x1) # Add 클래스 생성 과정이 감춰짐


# 사용
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = add(x0, x1)
print(y.data)

5


## Step 13
### 가변 길이 인수 (역전파 편)

- 다변수 함수(입력 변수가 여러개인 함수)에서는 편미분을 사용해서 기울기를 구함

In [8]:
class Add(Function):
  def forward(self, x0, x1):
    y = x0 + x1
    return y

  def backward(self, gy):
    return gy, gy  # 덧셈의 역전파는 상단에서의 미분값을 그대로 흘려보내는 것. y = x0 + x1 => ∂y/∂x0 = 1, ∂y/∂x1 = 1

In [9]:
# 기존 Variable 클래스
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError("{}은(는) 지원하지 않습니다.".format(type(data)))
        
        self.data = data
        self.grad = None 
        self.creator = None 

    def set_creator(self, func):
        self.creator = func
    
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        
        funcs = [self.creator]
        while funcs:
            f = funcs.pop() 
            x, y = f.input, f.output    # 1. 함수의 입출력을 얻는다
            x.grad = f.backward(y.grad) # 2. backward 메서드를 호출한다

            if x.creator is not None:
                funcs.append(x.creator)

# 기존 Variable 클래스  # 1. 에서는 입출력을 하나로 한정했다. 아래에서는, 여러 개의 입출력에 대응할 수 있도록 수정한다.

In [10]:
# 수정된 Variable 클래스
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError("{}은(는) 지원하지 않습니다.".format(type(data)))
        
        self.data = data
        self.grad = None 
        self.creator = None 

    def set_creator(self, func):
        self.creator = func
    
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]  # 1 함수의 출력에서 사용되는 grad 값만 저장
            gxs = f.backward(*gys)                       # 2 backward를 통해서 grad 값 저장
            if not isinstance(gxs, tuple):               # 3 grad 값이 한개의 원소만 있다면, 튜플로 변환
              gxs = (gxs,)
            
            for x, gx in zip(f.inputs, gxs):             # 4 함수의 인풋과 grad 값이 상응하므로, 반복문을 통해 각각의 인풋 값에 grad값 대입
              x.grad = gx

              if x.creator is not None:
                funcs.append(x.creator)

- 1. 출력 변수인 outputs에 담겨 있는 미분값들을 리스트에 담음
- 2. 함수 f의 역전파 호출 (list unpacking 사용)
- 3. gxs가 튜플이 아니라면, 튜플로 변환
- 4. 역전파로 전파되는 미분값을 Variable의 인스턴스 변수 grad에 저장. 여기서 f.inputs[i]의 미분 값은 gxs[i]임. 즉, 모든 Variable 인스턴스 각각에 알맞는 미분 값을 대입한 것

In [11]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.inputs[0].data # 수정 전 : x = self.input.data 
        gx = 2 * x * gy
        return gx

In [12]:
# z = x**2 + y**2 에서 ∂z/∂x(x=2.0), ∂z/∂y(y=3.0)을 계산하는 코드
x = Variable(np.array(2.0))
y = Variable(np.array(3.0))

z = add(square(x), square(y))
z.backward()
print(z.data) # 2**2 + 3**2
print(x.grad) # ∂z/∂x(x=2.0)
print(y.grad) # ∂z/∂y(y=3.0)

13.0
4.0
6.0


## Step 14
### 동일 변수 미분 에러 수정

In [13]:
x = Variable(np.array(3.0))
y = add(x, x) # 동일한 두 변수를 더함
print('y =', y.data) # y==3+3

y.backward()
print('x.grad =', x.grad) # d(2x)/dx = 2 이지만, 잘못된 값 출력

y = 6.0
x.grad = 1.0


In [16]:
class Variable:
  def __init__(self, data):
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError("{}은(는) 지원하지 않습니다.".format(type(data)))

    self.data = data
    self.grad = None 
    self.creator = None 

  def set_creator(self, func):
    self.creator = func
  
  def backward(self):
    if self.grad is None:
      self.grad = np.ones_like(self.data)
    
    funcs = [self.creator]
    while funcs:
      f = funcs.pop()
      gys = [output.grad for output in f.outputs]
      gxs = f.backward(*gys)
      if not isinstance(gxs, tuple):
        gxs = (gxs,)
      
      for x, gx in zip(f.inputs, gxs):
        # 수정 전: x.grad = gx  # 같은 변수를 사용하면 x.grad 값이 덮어 씌어짐.   
        if x.grad is None: # 덮어 씌울 x.grad 가 없다면, 기존과 동일하게 계산
          x.grad = gx
        else: # x.grad가 있다면, 미분 값을 더함
          x.grad = x.grad + gx # x.grad += gx # 라고 할 경우 inplace operation으로 후에 x.grad와 y.grad의 메모리 주소가 같아지는 경우가 발생함

        if x.creator is not None:
          funcs.append(x.creator)

In [17]:
x = Variable(np.array(3.0))
y = add(add(x, x), x)
y.backward()
print(x.grad) # x+x+x = 3x, d(3x)/dx = 3

3.0


In [18]:
# 문제점: 같은 인스턴스(여기서는 x)를 재사용하면 grad 값이 누적 더해짐

# 첫 번째 계산
x = Variable(np.array(3.0))
y = add(x, x) # y=2x
y.backward() 
print(x.grad) # dy/dx = d(2x)/dx = 2

# 두 번째 계산(같은 x를 사용하여 다른 계산을 수행)
y = add(add(x, x), x) # y=3x
y.backward()
print(x.grad) # dy/dx = d(3x)/dx = 3 인데, 5(2+3)가 나옴

2.0
5.0


In [20]:
# 위의 문제 해결
class Variable:
  def __init__(self, data):
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError("{}은(는) 지원하지 않습니다.".format(type(data)))

    self.data = data
    self.grad = None 
    self.creator = None 
  
  def set_creator(self, func):
    self.creator = func

  # grad 값을 초기화 해주는 함수 작성
  def cleargrad(self):
    self.grad = None

  def backward(self):
    if self.grad is None:
      self.grad = np.ones_like(self.data)
    
    funcs = [self.creator]
    while funcs:
      f = funcs.pop()
      gys = [output.grad for output in f.outputs]
      gxs = f.backward(*gys)
      if not isinstance(gxs, tuple):
        gxs = (gxs,)
      
      for x, gx in zip(f.inputs, gxs):
        if x.grad is None:
          x.grad = gx
        else: 
          x.grad = x.grad + gx 

        if x.creator is not None:
          funcs.append(x.creator)

# 첫 번째 계산
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print(x.grad) # 2.0

# 두 번째 계산(같은 x를 사용하여 다른 계산 수행)
x.cleargrad() # grad 값 초기화
y = add(add(x, x), x)
y.backward()
print(x.grad) # 3.0

2.0
3.0


## Step 16
### 복잡한 계산 그래프 (구현 편)

In [21]:
# 세대 'generation'을 이용하여 우선순위 결정
# 기존 입력 데이터가 0 세대, y쪽으로 갈수록 세대가 +1씩 됨
# 세대가 높을 수록 먼저 처리되어야 함. 즉, 세대가 우선순위
class Variable:
  def __init__(self, data):
    if data is not None:
      if not isinstance(data, np.ndarray):
        print(f'{type(data)}는 지원하지 않습니다.')
    
    self.data = data
    self.grad = None
    self.creator = None
    # 세대(우선순위)를 나타내는 generation 변수 추가
    self.generation = 0

  def set_creator(self, func):
    self.creator = func
    # '부모 세대 + 1' 만큼의 세대를 기억.
    self.generation = func.generation +1
  
  def cleargrad(self):
    self.grad = None

  def backward(self):
    if self.grad is None:
      self.grad = np.ones_like(self.data)

    funcs = [self.creator]
    while funcs:
      f = funcs.pop()
      gys = [output.grad for output in f.outputs]
      gxs = f.backward(*gys)
      if not isinstance(gxs, tuple):
        gxs = (gxs,)

      for x, gx in zip(f.input, gxs):
        if x.grad is None:
          x.grad = gx
        else:
          x.grad = x.grad + gx
      
      funcs.append(x.creator)

In [22]:
# Function의 generation 값 결정
class Function:
  def __call__(self, *inputs):
    xs = [x.data for x in inputs]
    ys = self.forward(*xs)
    if not isinstance(ys, tuple):
      ys = (ys,)
    outputs = [Variable(as_array(y)) for y in ys]

    # Function의 generation은 입력 변수에서 가장 큰 generation을 선택
    self.generation = max([x.generation for x in xs])
    for output in outputs:
      output.set_creator(self)
    self.inputs = inputs
    self.outputs = outputs
    return outputs if len(outputs) > 1 else outputs[0]

  def forward(self, xs):
    raise NotImplementedError()

  def backward(self, gys):
    raise NotImplementedError()

In [24]:
# generation이 높은 순으로 수행하는 예시
generations = [2, 0, 1, 4, 2]
funcs = []

# 임의로 function과 generation 생성
for g in generations:
  f = Function()
  f.generation = g
  funcs.append(f)

print('생성된 함수의 generation 값 :', [f.generation for f in funcs])

funcs.sort(key=lambda x:x.generation) # genetion을 key 값으로 함수 내림차순 정렬
print('내림차순된 함수의 generation 값 :', [f.generation for f in funcs])

f = funcs.pop() # 리스트(스택)에서 가장 위(큰 인덱스)의 값을 꺼냄
print(f.generation) # 꺼낸 함수의 generation 값 출력. # generation이 가장 높은 것이 꺼내졌음을 볼 수 있음

생성된 함수의 generation 값 : [2, 0, 1, 4, 2]
내림차순된 함수의 generation 값 : [0, 1, 2, 2, 4]
4


In [ ]:
# generation이 높은 순으로 backward 수행
class Variable:
  def __init__(self, data):
    if not data is None:
      if isinstance(data, np.ndarray):
        print(f'{format(data)}는 지원하지 않는 형식입니다.')
      
    self.data = data
    self.grad = None
    self.creator = None
    self.generation = 0

    def set_creator(self, func):
      self.creator = func
      self.generation = func.generation +1

    def cleargrad(self):
      self.grad = None

    def backward(self, dy):
      if self.grad is None:
        self.grad = np.ones_like(self.data)
      
      # 수정 전: funcs = [self.creator]
      funcs = [] 
      seen_set = set() # 집합 자료형 선언, 이미 처리한 함수인지 확인하기 위함
      # 처리하지 않은 함수만 funcs 함수에 append 함
      def add_func(f):
        if f not in seen_set:
          funcs.append(f)
          seen_set.add(f)
      add_func(self.creator) # 부모함수 추가

      while funcs:
        f = funcs.pop()
        gys = [output.grad for output in f.outputs]
        gxs = f.backward(*gys)
        if not isinstance(gxs, tuple):
          gxs = (gxs,)
        
        for x, gx in zip(f.inputs, gxs):
          if x.grad is None:
            x.grad = gx
          else: 
            x.grad = x.grad + gx 

          if x.creator is not None:
            add_func(x.creator) # 수정 전: funcs.append(x.creator)